# Impact of rainfall temporal distribution on flooding in Lin Dyke catchment


## Project Summary

Research suggests that the temporal distribution of rainfall within a storm event can influence the nature and severity of flooding resulting from that storm event....

Testing this using synthetic rainfall events derived from FEH/ReFH2. Standard profiles usually have one peak, testing three methods for splitting the total rainfall amount normally concentrated in this one peak into multiple peaks 

Analysing impact on the extent, depth and velocity of flooding, and the spatial distribution of these variables over the catchment

### Code initialisation

In [4]:
clusters['Duration'].unique()

array(['[19:25 - Inf) hr', '[6:45 - 19:25) hr', '[2:10 - 6:45) hr',
       '[0.25 - 2:10) hr'], dtype=object)

In [1]:
import os
from pathlib import Path
# from PIL import Image
from my_functions_new import *

# Define whether to filter out values <0.1
remove_little_values = True
model_directory = '../../../../FloodModelling/MeganModel_new/'

In [2]:
methods = ['6h_single-peak', '6h_divide-time', '6h_subpeak-timing', '6h_max-spread', 'Cluster1', 'Cluster2', 'Cluster3', 
           'Cluster4', 'Cluster5', 'Cluster6', 'Cluster7', 'Cluster8', 'Cluster9','Cluster10', 'Cluster11', 'Cluster12',
           'Cluster13', 'Cluster14', 'Cluster15']  
short_ids = ['6h_sp', '6h_dt', '6h_spt', '6h_ms', '6h_c1','6h_c2','6h_c3','6h_c4', '6h_c5', '6h_c6','6h_c7',
             '6h_c8','6h_c9','6h_c10', '6h_c11', '6h_c12','6h_c13','6h_c14','6h_c15',]   

In [3]:
fps = []
for method_num, short_id in enumerate(short_ids):
    fp = model_directory + "{}/{} (Max).Resampled.Terrain.tif".format(short_id, '{}')
    fps.append(fp)

In [4]:
breaks_difference = np.array([-0.1, 0.1, 0.3, 100])

#### Read in rasters containing depths and extents using each of the methods, and store these in a dictionary
#### Create versions of depth/velocity rasters which are classified into depth/velocity categories

In [3]:
def do_processing (variable_name):    
    
    ######################################################################################
    # Specify breaks and labels for this variable
    ######################################################################################
    if variable_name == 'Depth':
        breaks = np.array([0, 0.3, 0.6, 1.2, 100])  
        labels = ['<=0.3m', '0.3-0.6m', '0.6-1.2m', '>1.2m']
        classified_colors_list = [mpl.cm.Blues(0.2), mpl.cm.Blues(0.5), mpl.cm.Blues(0.7),"navy"]
    elif variable_name == 'Velocity':
        breaks = np.array([0,0.25,0.5,2,100])
        labels = ["<=0.25m/s", "0.25-0.5m/s", "0.5-2m/s", ">2m/s"]
        classified_colors_list = [mpl.cm.cool(0.3), mpl.cm.cool(0.5), mpl.cm.cool(0.7), mpl.cm.cool(0.9)]
        
    ######################################################################################
    # For the single peak scenarios:
        # Classify depth/velocity rasters into depth/velocity bins
        # Save as raster
        # Plot this raster, and save the output
    ######################################################################################
    single_peak, out_meta = prepare_rainfall_scenario_raster(fps[0].format(variable_name), remove_little_values)
    singlepeak_classified = classify_raster(single_peak, breaks)
    fp_for_classified_raster = fps[0].format(variable_name).replace('{} (Max).Resampled.Terrain'.format(variable_name),'{}_classified'.format(variable_name))
    save_array_as_raster(singlepeak_classified, fp_for_classified_raster, out_meta) 
    plot_classified_raster(fp_for_classified_raster, labels, classified_colors_list)
    
    ######################################################################################
    # For the other rainfall scenarios:
        # Classify depth/velocity rasters into depth/velocity bins
            # Save as raster
            # Plot this raster, and save the output
        # Find the difference between depth/velocity rasters for this scenario and the single peak scenario:
            # Save this as a raster
            # Plot this raster and save the output
        # Reclassify the difference rasters into bins:
            # Save this as a raster
            # Plot this raster and save the output
        # Reclassify the difference rasters into positive, negative or neutral differences:
            # Save this as a raster
            # Plot this raster and save the output
    ######################################################################################
    for fp in fps[1:]:
        # Create directory to store results
        # processing_dir = "HecRas_Processing/" + fp.split('/')[1]
        # Path(processing_dir).mkdir(parents=True, exist_ok=True)
    
        # Classify depth/velocity rasters into depth/velocity bins
        this_scenario_raster = prepare_rainfall_scenario_raster(fp.format(variable_name), remove_little_values)[0]
        this_scenario_classified = classify_raster(this_scenario_raster, breaks)
        fp_for_classified_raster = fp.replace('{} (Max).Resampled.Terrain','{}_classified'.format(variable_name) )
        save_array_as_raster(this_scenario_classified, fp_for_classified_raster, out_meta) 
        plot_classified_raster(fp_for_classified_raster, labels, classified_colors_list)    

        # Find difference between depth/velocity rasters from this method, and the single peak method
        difference_raster = single_peak - this_scenario_raster
        # fp_for_difference_raster = "Arcpy/{}_singlepeak_{}_diff.tif".format(variable_name, rainfall_scenario_name)
        # save_array_as_raster(singlepeak_classified, fp_for_difference_raster, out_meta) 
        # plot_difference(variable_name, rainfall_scenario_name, labels, 'YlGnBu') 

        # Reclassify the difference rasters to categories 
        classified_difference_raster = classify_raster (difference_raster, breaks_difference)
        fp_for_classified_diff_raster = fp.replace('{} (Max).Resampled.Terrain','{}_difffromsinglepeak_classified'.format(variable_name))
        save_array_as_raster(classified_difference_raster, fp_for_classified_diff_raster, out_meta) 
        plot_difference_levels(fp_for_classified_diff_raster, labels)    

        # Reclassify the difference rasters to represent whether value is positive or negative 
        pos_neg_depth_diff = np.where((np.isnan(difference_raster)), np.nan, 
                                  np.where((difference_raster > 0), 0,
                                    np.where((difference_raster == 0), 1,
                                        np.where((difference_raster < 0), 2, 500))))
        fp_for_posneg_diff_raster = fp.replace('{} (Max).Resampled.Terrain','{}_difffromsinglepeak_posneg'.format(variable_name) )
        save_array_as_raster(pos_neg_depth_diff, fp_for_posneg_diff_raster, out_meta) 
        plot_difference_levels_pos_neg(fp_for_posneg_diff_raster)  

        
# Run the function  
do_processing('Depth')
do_processing('Velocity')